In [4]:
import numpy as np

In [5]:
# extract api call from txt file
ransomware = ''
table = dict()
number = 0
f = open('./ransomware.txt', 'r')
for lines in f:
    try:
        temp = lines.split('\t')
        function = temp[2].split(' ')[0]
        ransomware += (function + ' ') 
        if not(function in table.keys()):
            table[function] = dict()
    except:
        pass 

record = []
decrypt = ''
decrypt_table = dict()
f = open('./decrypt.txt', 'r')
for lines in f:
    try:
        temp = lines.split('\t')
        function = temp[2].split(' ')[0]
        if('::' in function):
            function = function.split('::')[1]
        record.append(function)
        decrypt += (function + ' ') 
        if not(function in decrypt_table.keys()):
            decrypt_table[function] = dict()
    except:
        pass 
ransomware = [ransomware[4:]]
decrypt = [decrypt[5:]]
data = [ransomware[0],decrypt[0]]

del table['API']   
del table['']
del decrypt_table['API\n']
del decrypt_table['']

# Method 1

In [6]:
# use countervectorizer to calculate the weigth of edge
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(lowercase = False ,ngram_range=(2, 2))
ransomware_occurence_times = vectorizer.fit_transform(ransomware)
ransomware_edge_kind = vectorizer.get_feature_names()

vectorizer = CountVectorizer(lowercase = False ,ngram_range=(2, 2))
decrypt_occurence_times = vectorizer.fit_transform(decrypt)
decrypt_edge_kind = vectorizer.get_feature_names()

vectorizer = CountVectorizer(lowercase = False ,ngram_range=(2, 2))
data_occurence = vectorizer.fit_transform(data)
data_edge_kind = vectorizer.get_feature_names()

In [7]:
data_occurence = data_occurence.toarray()

In [8]:
# use dictionary to record the relation between node
for i in data_edge_kind:
    try:
        start = i.split(' ')[0]
        end = i.split(' ')[1]
        temp = data_occurence[0,data_edge_kind.index(i)]
        if(temp):
            table[start][end] = temp
    except:
        pass
    
for i in data_edge_kind:
    try:
        start = i.split(' ')[0]
        end = i.split(' ')[1]
        temp = data_occurence[1,data_edge_kind.index(i)]
        if(temp):
            decrypt_table[start][end] = temp
    except:
        pass

In [10]:
total_edge = sorted(list(set(list(decrypt_table.keys()) + list(table.keys()))))

In [11]:
# create the adjacency matrix 
ransomware_matrix = np.zeros((len(total_edge),len(total_edge)))
for i in list(table.keys()):
    for j in list(table[i].keys()):
        try:
            ransomware_matrix[total_edge.index(i),total_edge.index(j)] = table[i][j]
        except:
            print(i,j)
            
decrypt_matrix= np.zeros((len(total_edge),len(total_edge)))
for i in list(decrypt_table.keys()):
    for j in list(decrypt_table[i].keys()):
        try:
            decrypt_matrix[total_edge.index(i),total_edge.index(j)] = decrypt_table[i][j]
        except:
            print(i,j)

In [13]:
decrypt_matrix

array([[   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0., 7556.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.]])

In [14]:
# calculate the eigenvalue and eigenvector 
ran_val, ran_vec = np.linalg.eig(ransomware_matrix)
dec_val, dec_vec = np.linalg.eig(decrypt_matrix)

In [15]:
ran_vector = ran_vec[ran_val.tolist().index(max(ran_val))]
dec_vector = dec_vec[dec_val.tolist().index(max(dec_val))]

In [16]:
# calculate the Euclidean distance
result = np.sqrt(np.sum(ran_vector - dec_vector))

In [36]:
# function to calculate the cosine similarity
def cosine_similarity(x,y):
    num = x.dot(y.T)
    denom = np.linalg.norm(x) * np.linalg.norm(y)
    return num / denom

In [25]:
# calculate the cosine similarity
cosine_similarity(ran_vector, dec_vector)

(0.002487991804125693-0.03731834688382809j)

# Method 2

In [18]:
import networkx as nx

In [19]:
# Generate the graph of two file
g_ran = nx.Graph()
for i in list(table.keys()):
    for j in list(table[i].keys()):
        g_ran.add_edge(i,j, weight = table[i][j])

        
g_dec = nx.Graph()
for i in list(decrypt_table.keys()):
    for j in list(decrypt_table[i].keys()):
        g_dec.add_edge(i,j, weight = decrypt_table[i][j])

In [37]:
# the function to generate the feature 
def get_density(G):

    degree = {d[0]: d[1] for d in G.degree(G.nodes())}
    density = (sum(degree.values())/(len(degree)-1)) / len(degree)

    return [density]

def shortest_path(G):
    shortest_path = nx.average_shortest_path_length(G)
    return [shortest_path]

def closeness_centrality(G):

    List = list(nx.closeness_centrality(G).values())
    closeness_centrality = []
    closeness_centrality.append(np.mean(List))
    closeness_centrality.append(np.max(List))
    closeness_centrality.append(np.min(List))
    closeness_centrality.append(np.median(List))
    closeness_centrality.append(np.std(List))
    return closeness_centrality

def degree_centrality(G):

    List = list(nx.degree_centrality(G).values())
    degree_centrality = []
    degree_centrality.append(np.mean(List))
    degree_centrality.append(np.max(List))
    degree_centrality.append(np.min(List))
    degree_centrality.append(np.median(List))
    degree_centrality.append(np.std(List))

    return degree_centrality

def node_and_edge(G):
    return [len(G.nodes()), len(G.edges())]

def generate_vec(G):
    vec = node_and_edge(G) + get_density(G) + shortest_path(G) + closeness_centrality(G) + degree_centrality(G)
    return(vec)

In [22]:
vec_gan = np.array(generate_vec(g_ran))
vec_dec = np.array(generate_vec(g_dec))

In [24]:
# calculate the cosine similarity
cosine_similarity(vec_dec,vec_gan)

0.9990533007920036

## Compare the feature between two graph

In [26]:
degree_centrality(g_dec)

[0.017927730514737474,
 0.26450116009280744,
 0.002320185614849188,
 0.009280742459396751,
 0.03053233451413935]

In [27]:
degree_centrality(g_ran)

[0.01827660458615088,
 0.30739299610894943,
 0.0019455252918287938,
 0.007782101167315175,
 0.03270408812562848]

In [28]:
closeness_centrality(g_dec)

[0.35407609304112,
 0.53875,
 0.2260094389092816,
 0.3512632436837816,
 0.04790165066469869]

In [29]:
closeness_centrality(g_ran)

[0.3707564302836897,
 0.5717463848720801,
 0.25623130608175476,
 0.3660968660968661,
 0.0479681902241983]

In [30]:
shortest_path(g_dec)

[2.8751718656011]

In [31]:
shortest_path(g_ran)

[2.7412564693438104]

In [32]:
get_density(g_dec)

[0.017927730514737474]

In [33]:
get_density(g_ran)

[0.018276604586150883]

In [34]:
node_and_edge(g_dec)

[432, 1669]

In [35]:
node_and_edge(g_ran)

[515, 2419]

In [38]:
import pickle

### Save graph

In [40]:
pickle.dump(g_ran, open('./ransomware.pkl','wb'))

In [44]:
pickle.dump(g_dec, open('./decrypt.pkl','wb'))